In [1]:
import pandas as pd

# Load the data
file_path = 'saleData.csv'
sales_data = pd.read_csv(file_path)

# Display the first few rows of the dataframe and its info to understand its structure
sales_data.head(), sales_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   userid        80000 non-null  int64  
 1   productname   80000 non-null  object 
 2   saleDate      80000 non-null  object 
 3   pricePerItem  80000 non-null  int64  
 4   qty           80000 non-null  int64  
 5   discount      80000 non-null  float64
 6   totalPrice    80000 non-null  int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 4.3+ MB


(   userid                             productname    saleDate  pricePerItem  \
 0       1    COLMANS N/COCONUT WATER VINEGR 350ML  2022-01-01           223   
 1       1                 MCCURRIE FENUGREEK 100G  2022-01-02           151   
 2       1         SPINNER CAFFEINATED DRINK 250ML  2022-01-02           344   
 3       1  RITZBURY SUGAR FREE DARK CHOCOLATE 45G  2022-01-02            99   
 4       1        HARISCHANDRA MASALA PAPPADAM 70G  2022-01-02           142   
 
    qty  discount  totalPrice  
 0    4       0.0         892  
 1    3       0.0         453  
 2    4       0.0        1376  
 3    4       0.0         396  
 4    4       0.0         568  ,
 None)

In [2]:
# Convert 'saleDate' to datetime format
sales_data['saleDate'] = pd.to_datetime(sales_data['saleDate'])

# Create the weighted sales feature
sales_data['weightedSales'] = 0.6 * sales_data['qty'] + 0.4 * sales_data['totalPrice']

# Check the updated DataFrame
sales_data.head()

,userid,productname,saleDate,pricePerItem,qty,discount,totalPrice,weightedSales
0,1,COLMANS N/COCONUT WATER VINEGR 350ML,2022-01-01,223,4,0.0,892,359.2
1,1,MCCURRIE FENUGREEK 100G,2022-01-02,151,3,0.0,453,183.0
2,1,SPINNER CAFFEINATED DRINK 250ML,2022-01-02,344,4,0.0,1376,552.8
3,1,RITZBURY SUGAR FREE DARK CHOCOLATE 45G,2022-01-02,99,4,0.0,396,160.8
4,1,HARISCHANDRA MASALA PAPPADAM 70G,2022-01-02,142,4,0.0,568,229.6


In [3]:
import pandas as pd

# Convert 'saleDate' to datetime format and create a 'ds' column for Prophet
sales_data['saleDate'] = pd.to_datetime(sales_data['saleDate'])
sales_data['ds'] = sales_data['saleDate'].dt.to_period('M').dt.start_time

# Aggregate sales data by product and month
monthly_sales = sales_data.groupby(['productname', 'ds']).agg(
    y=('weightedSales', 'sum')).reset_index()

# Sort the data by 'ds' for correct chronological splitting
monthly_sales = monthly_sales.sort_values(by='ds')

# Split the data into training and testing sets (80% training, 20% testing)
split_index = int(len(monthly_sales) * 0.8)
train_data = monthly_sales.iloc[:split_index]
test_data = monthly_sales.iloc[split_index:]


In [4]:
from prophet import Prophet

models = {}

# Train a model for each product
for product in train_data['productname'].unique():
    product_data = train_data[train_data['productname'] == product]
    model = Prophet(yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=False)
    model.fit(product_data[['ds', 'y']])
    models[product] = model


09:21:45 - cmdstanpy - INFO - Chain [1] start processing
09:21:46 - cmdstanpy - INFO - Chain [1] done processing
09:21:47 - cmdstanpy - INFO - Chain [1] start processing
09:21:47 - cmdstanpy - INFO - Chain [1] done processing
09:21:47 - cmdstanpy - INFO - Chain [1] start processing
09:21:48 - cmdstanpy - INFO - Chain [1] done processing
09:21:48 - cmdstanpy - INFO - Chain [1] start processing
09:21:50 - cmdstanpy - INFO - Chain [1] done processing
09:21:50 - cmdstanpy - INFO - Chain [1] start processing
09:21:52 - cmdstanpy - INFO - Chain [1] done processing
09:21:52 - cmdstanpy - INFO - Chain [1] start processing
09:21:54 - cmdstanpy - INFO - Chain [1] done processing
09:21:54 - cmdstanpy - INFO - Chain [1] start processing
09:22:08 - cmdstanpy - INFO - Chain [1] done processing
09:22:08 - cmdstanpy - INFO - Chain [1] start processing
09:22:23 - cmdstanpy - INFO - Chain [1] done processing
09:22:23 - cmdstanpy - INFO - Chain [1] start processing
09:22:27 - cmdstanpy - INFO - Chain [1]

OSError: [Errno 28] No space left on device

In [5]:
test_results = pd.DataFrame()

# Loop through each model and its corresponding product
for product, model in models.items():
    product_data = test_data[test_data['productname'] == product]
    if not product_data.empty:
        forecast = model.predict(product_data[['ds']])
        forecast['productname'] = product
        test_results = pd.concat([test_results, forecast[['ds', 'productname', 'yhat', 'yhat_lower', 'yhat_upper']]], ignore_index=True)


In [6]:
from sklearn.metrics import mean_squared_error
import numpy as np

# Calculate RMSE for each product
rmse_results = {}

rmse_items = 0;
rmse_value = 0;

for product, forecast in test_results.groupby('productname'):
    true_values = test_data[(test_data['productname'] == product) & (test_data['ds'].isin(forecast['ds']))]['y']
    rmse = np.sqrt(mean_squared_error(true_values, forecast['yhat']))
    rmse_value = rmse_value + rmse
    rmse_items = rmse_items + 1
    rmse_results[product] = rmse

# Print RMSE results
print(rmse_results)

# Print RMSE average
print(rmse_value / rmse_items)


{'7 UP 500ML PET': 7231.1558437479525, 'ASTRA FAT SPREAD 100G': 20534.031278729442, 'ASTRA FAT SPREAD 250G TUB': 324380.39777100936, 'ATLAS CHOOTY 11 3IN PAC ASSORTED': 2008.6216674224565, 'ATLAS CHOOTY 11 3IN PAC BLUE': 801.6573923151661, 'ATLAS CR BOOK 120 PGS SINGLE': 7362.414179738772, 'ATLAS CR BOOK 80 PGS': 7428.876901075293, 'ATLAS RULER CLEAR 6': 777.6174423823071, 'ATLAS TAPE EASY LARGE': 1885.1877150333892, 'AVOCADO JUICE': 21352.353794161263, 'AXE OIL 5ML': 13592.823482140635, 'BABY CHERAMY HERBAL CREAM 100ML': 17195.476774773266, 'BABY CHERAMY PAPER COTTON BUDS 100S': 3001.855203373957, 'BABY CHERAMY SOAP FLORAL 5*75G': 8809.53918704925, 'BABY CHERAMY SOAP POKURUWA/SADUN 5IN1': 73827.41910790939, 'BAIRAHA WHOLE CHICKEN SKINLESS': 15158.6745329559, 'BANANA - CAVENDISH': 14254.30314358023, 'BARAKA ACT FAST 5ML': 23232.038139564884, 'BEAM PAAWATTA SYRUP 100ML': 26737.184738091502, 'BEETROOT': 5838.336573210425, 'BFRESH DRINK NARANG 200ML': 3056.7796006886674, 'BIC EASY 2 PRECI

In [7]:
def calculate_mape(actual, predicted):
    # Remove zero values to avoid division by zero error
    mask = actual != 0
    return (np.abs(actual[mask] - predicted[mask]) / actual[mask]).mean() * 100

# Initialize a dictionary to store MAPE for each product
mape_results = {}
mape_items = 0;
mape_total_value=0;

for product, forecast in test_results.groupby('productname'):
    true_values = test_data[(test_data['productname'] == product) & (test_data['ds'].isin(forecast['ds']))]['y']
    predicted_values = forecast['yhat']
    mape = calculate_mape(true_values.values, predicted_values.values)
    mape_items = mape_items+1;
    mape_total_value = mape_total_value + mape
    mape_results[product] = mape

# Print MAPE results
print(mape_results)

# Print Mape Average
print(mape_total_value/mape_items)


{'7 UP 500ML PET': 543.3234777741155, 'ASTRA FAT SPREAD 100G': 1609.7484531732744, 'ASTRA FAT SPREAD 250G TUB': 7835.920092031008, 'ATLAS CHOOTY 11 3IN PAC ASSORTED': 337.46965176349414, 'ATLAS CHOOTY 11 3IN PAC BLUE': 124.52486588870669, 'ATLAS CR BOOK 120 PGS SINGLE': 626.4116181280218, 'ATLAS CR BOOK 80 PGS': 234.00797472560012, 'ATLAS RULER CLEAR 6': 488.71566986587584, 'ATLAS TAPE EASY LARGE': 427.15299989214486, 'AVOCADO JUICE': 801.0106383109028, 'AXE OIL 5ML': 433.05289551578755, 'BABY CHERAMY HERBAL CREAM 100ML': 608.2964386603795, 'BABY CHERAMY PAPER COTTON BUDS 100S': 229.5451550971264, 'BABY CHERAMY SOAP FLORAL 5*75G': 201.57511096980812, 'BABY CHERAMY SOAP POKURUWA/SADUN 5IN1': 886.4186537757744, 'BAIRAHA WHOLE CHICKEN SKINLESS': 187.17859882295457, 'BANANA - CAVENDISH': 506.1358388047351, 'BARAKA ACT FAST 5ML': 723.0495909795408, 'BEAM PAAWATTA SYRUP 100ML': 803.9692955098051, 'BEETROOT': 358.2446880576365, 'BFRESH DRINK NARANG 200ML': 84.37538700816067, 'BIC EASY 2 PRECI

In [8]:
import os

# Define the directory path for models with high accuracy
high_accuracy_dir = 'models/highAccuracy/product'

# Create the directory if it does not already exist
os.makedirs(high_accuracy_dir, exist_ok=True)


In [9]:
import os
import pickle
import re

# Function to sanitize filenames by replacing or removing invalid characters
def sanitize_filename(name):
    # Replace invalid characters with an underscore or another suitable character
    return re.sub(r'[<>:"/\\|?*]', '_', name)  # Add any other characters that might cause issues

# MAPE threshold for good accuracy
good_accuracy_threshold = 100

# Export models with MAPE less than the threshold
for product, mape in mape_results.items():
    if mape < good_accuracy_threshold:
        model = models[product]
        safe_product_name = sanitize_filename(product)
        filename = os.path.join(high_accuracy_dir, f'model_{safe_product_name}.pkl')
        with open(filename, 'wb') as f:
            pickle.dump(model, f)
            print(f"Exported model for {product} with MAPE {mape}%")



Exported model for BFRESH DRINK NARANG 200ML with MAPE 84.37538700816067%
Exported model for CHELLO PASTURIZED MILK 1L with MAPE 59.59562911018595%
Exported model for CHICKEN DRUMSTICKS SKIN ON with MAPE 69.11912123727598%
Exported model for CHILLED PIZZA LARGE - ROAST CHICKEN with MAPE 96.66974366040479%
Exported model for CLOGARD FRESH GEL T/P LEMONGRASS 120G with MAPE 39.87020492491773%
Exported model for DATE CROWN DATES KHALAS THERMO 250G with MAPE 81.73407592296262%
Exported model for DELMEGE SOYA MEAT MUTTON 90G with MAPE 72.73465398474899%
Exported model for DIABETASOL NUTRI POWDER VANILLA 180G with MAPE 59.88475029548923%
Exported model for DIMO 12W DAY PIN BUY 2 SAVE 300 with MAPE 20.85519486674591%
Exported model for DOVE SHAMPO OXYGEN MOISTURE 180ML with MAPE 97.67416623582066%
Exported model for KEELLS WAFERS CHOCOLATE 100G with MAPE 99.55505986377489%
Exported model for KIZZ SPARKLING APPLE DRINK 215ML with MAPE 50.75162384618884%
Exported model for NATURE CHILLIE PIECES 